In [1]:
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

In [2]:
# 사용할 모델 식별자 정의 (Hugging Face Model ID)
# Qwen3-30B-A3B-FP8: MoE 아키텍처 기반이며, FP8(8-bit Floating Point)로 양자화된 모델
model_name = "Qwen/Qwen3-30B-A3B-FP8"

In [3]:
# 1. 토크나이저 초기화 (Tokenizer Initialization)
# AutoTokenizer: 입력 텍스트(String)를 모델이 처리 가능한 수치 데이터인 'Token ID(Integer)'로 변환하는 전처리 모듈입니다.
# - 역할: 모델의 고유 어휘 집합(Vocabulary)에 매핑된 인덱스를 반환하며, 모델은 이 인덱스를 통해 임베딩 레이어(Embedding Layer)의 벡터 값을 참조합니다.
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
# 2. vLLM 추론 엔진 초기화 (Inference Engine Initialization)
# LLM 클래스: 고성능 추론을 위한 vLLM 엔진 인스턴스를 생성합니다.
llm = LLM(
    model=model_name,
    
    # [데이터 정밀도 설정]
    # "auto": 모델 설정 파일(config.json)을 감지하여 최적의 정밀도를 로드합니다.
    # 본 모델은 FP8 형식이므로, 이를 인식하여 RTX 6000 Ada 등의 GPU에서 Tensor Core 가속을 활성화합니다.
    dtype="auto",

    # [분산 처리 설정]
    # tensor_parallel_size: 모델의 가중치 행렬(Weight Matrix)을 몇 개의 GPU에 분할하여 적재할지 결정합니다.
    # 1로 설정 시 단일 GPU에서 모든 연산을 수행합니다.
    tensor_parallel_size=1,

    # [메모리 관리 설정]
    # gpu_memory_utilization: GPU VRAM의 할당 비율(0.0 ~ 1.0)을 설정합니다.
    # 0.95: 가중치 로드 후 남은 VRAM의 95%를 KV Cache 블록으로 사전 할당합니다.
    # 이는 긴 컨텍스트 처리를 위한 메모리 공간을 확보하여 Throughput을 최대화하기 위함입니다.
    gpu_memory_utilization=0.95 # VRAM 48GB를 꽉 채워 쓰도록 설정
)

INFO 12-02 09:59:08 [utils.py:253] non-default args: {'gpu_memory_utilization': 0.95, 'disable_log_stats': True, 'model': 'Qwen/Qwen3-30B-A3B-FP8'}
INFO 12-02 09:59:09 [model.py:631] Resolved architecture: Qwen3MoeForCausalLM
INFO 12-02 09:59:09 [model.py:1745] Using max model len 40960
INFO 12-02 09:59:10 [scheduler.py:216] Chunked prefill is enabled with max_num_batched_tokens=8192.
(EngineCore_DP0 pid=28616) INFO 12-02 09:59:11 [core.py:93] Initializing a V1 LLM engine (v0.11.2) with config: model='Qwen/Qwen3-30B-A3B-FP8', speculative_config=None, tokenizer='Qwen/Qwen3-30B-A3B-FP8', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=40960, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=fp8, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=Struc

Loading safetensors checkpoint shards:   0% Completed | 0/7 [00:00<?, ?it/s]


(EngineCore_DP0 pid=28616) INFO 12-02 09:59:24 [default_loader.py:314] Loading weights took 11.04 seconds
(EngineCore_DP0 pid=28616) INFO 12-02 09:59:25 [gpu_model_runner.py:3338] Model loading took 29.0445 GiB memory and 12.163751 seconds


(EngineCore_DP0 pid=28616) /home/jovyan/work/skku_venv/lib/python3.12/site-packages/torch/_dynamo/variables/functions.py:1692: UserWarning: Dynamo detected a call to a `functools.lru_cache`-wrapped function. Dynamo ignores the cache wrapper and directly traces the wrapped function. Silent incorrectness is only a *potential* risk, not something we have observed. Enable TORCH_LOGS="+dynamo" for a DEBUG stack trace.
(EngineCore_DP0 pid=28616)   torch._dynamo.utils.warn_once(msg)


(EngineCore_DP0 pid=28616) INFO 12-02 09:59:37 [backends.py:631] Using cache directory: /home/jovyan/.cache/vllm/torch_compile_cache/b003b827d1/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=28616) INFO 12-02 09:59:37 [backends.py:647] Dynamo bytecode transform time: 12.57 s
(EngineCore_DP0 pid=28616) INFO 12-02 09:59:43 [backends.py:210] Directly load the compiled graph(s) for dynamic shape from the cache, took 4.581 s
(EngineCore_DP0 pid=28616) WARNING 12-02 09:59:45 [fp8_utils.py:777] Using default W8A8 Block FP8 kernel config. Performance might be sub-optimal! Config file not found at /home/jovyan/work/skku_venv/lib/python3.12/site-packages/vllm/model_executor/layers/quantization/utils/configs/N=5120,K=2048,device_name=NVIDIA_RTX_6000_Ada_Generation,dtype=fp8_w8a8,block_shape=[128,128].json
(EngineCore_DP0 pid=28616) WARNING 12-02 09:59:46 [fp8_utils.py:777] Using default W8A8 Block FP8 kernel config. Performance might be sub-optimal! Config file not found at /home/

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:08<00:00,  6.00it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:04<00:00,  7.49it/s]


(EngineCore_DP0 pid=28616) INFO 12-02 10:00:03 [gpu_model_runner.py:4244] Graph capturing finished in 14 secs, took 0.92 GiB
(EngineCore_DP0 pid=28616) INFO 12-02 10:00:03 [core.py:250] init engine (profile, create kv cache, warmup model) took 38.25 seconds
INFO 12-02 10:00:04 [llm.py:352] Supported tasks: ['generate']


In [5]:
# 3. 샘플링 설정
# SamplingParams: 다음 토큰을 Sampling할 때 사용할 확률적 알고리즘의 하이퍼파라미터를 정의합니다.
sampling_params = SamplingParams(
    # [확률 분포 조절]
    # temperature: Softmax 출력 확률 분포의 Flatness 정도를 조절합니다.
    # 0.7: 분포를 적당히 평탄하게 하여, 가장 높은 확률의 토큰 외에도 차순위 토큰이 선택될 가능성을 부여합니다 (생성 다양성 확보).
    temperature=0.7, 

    # [후보군 제한]
    # top_p: Cumulative Probability가 80%에 도달하는 상위 토큰 집합 내에서만 샘플링을 수행합니다.
    # 확률이 매우 낮은 토큰의 생성을 배제하여 텍스트의 품질을 유지합니다.
    top_p=0.8, 

    # [반복 억제]
    # repetition_penalty: 이미 생성된 토큰의 로짓(Logit) 값을 인위적으로 낮추어 재선택 확률을 감소시킵니다.
    # 1.05: 5%의 페널티를 부여하여, 동일한 문장이 무한 반복(Infinite Loop)되는 현상을 방지합니다.
    repetition_penalty=1.05,

    # [생성 길이 제한]
    # max_tokens: 모델이 생성할 수 있는 최대 신규 토큰 수(Budget)를 제한합니다.
    max_tokens=4096 # 최대 32768
)

In [6]:
# 4. 프롬프트 엔지니어링 (Prompt Construction)
prompt = "성균관대학교가 최고의 대학교인 이유 한 가지 말해줘."

# [대화형 구조 정의]
# 리스트 내 딕셔너리 구조를 사용하여 시스템과 사용자의 역할을 명시합니다.
messages = [
    {"role": "system", "content": "You are a helpful assistant."}, # System instruction은 여기에 적으면 됩니다.
    {"role": "user", "content": prompt}
]

# [채팅 템플릿 적용]
# apply_chat_template: 구조화된 메시지 리스트를 모델 학습 시 사용된 특수 토큰(Special Tokens)이 포함된 단일 문자열로 변환합니다.
# 예: "<|im_start|>system\nYou are...<|im_end|>\n<|im_start|>user\nTell me...<|im_end|>\n<|im_start|>assistant"
# tokenize=False: 토큰 ID가 아닌 가공된 텍스트 문자열(Raw String)을 반환받습니다 (vLLM 입력용).
# add_generation_prompt=True: 모델이 답변을 시작할 수 있도록 어시스턴트의 시작 토큰을 프롬프트 끝에 자동으로 추가합니다.
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [7]:
outputs = llm.generate([text], sampling_params)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [8]:
import re

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text.strip()

    # 변수 초기화
    thinking_content = None
    final_response = generated_text

    # Case 1: </think> (닫는 태그)가 명확히 있는 경우 -> 가장 일반적
    if "</think>" in generated_text:
        parts = generated_text.split("</think>")
        
        # 앞부분: <think> 태그 삭제 후 '사고' 내용으로 저장
        thinking_content = parts[0].replace("<think>", "").strip()
        
        # 뒷부분: 실제 답변
        if len(parts) > 1:
            final_response = parts[1].strip()
        else:
            final_response = "" # 생각만 하고 답변은 아직 안 나온 경우

    print(f"입력: {prompt}")
    print()

    if thinking_content:
        print(f"사고: {thinking_content}")
        print()

    if final_response:
        print(f"답변: {final_response}")
    
    print("-" * 50) # 구분선

입력: <|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
성균관대학교가 최고의 대학교인 이유 한 가지 말해줘.<|im_end|>
<|im_start|>assistant


사고: Okay, the user is asking why Sungkyunkwan University is the best, and they want just one reason. Let me think about what makes SKKU stand out.

First, I know that SKKU is one of the oldest universities in Korea, founded in 1398. That's a long history, which gives it a lot of prestige. But is that the main reason people consider it the best? Maybe not just the age, but how it's maintained its reputation over time.

Another point is its academic excellence. SKKU has strong programs in various fields, especially in engineering, business, and humanities. They might have good research facilities and faculty. Also, their international partnerships and exchange programs could be a factor. But again, is that the top reason?

Then there's the campus and location. The main campus is in Seoul, which is a big city with lots of opportunities. The archite

In [9]:
# 멀티턴 대화 해보기
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import re

class QwenVLLMChatbot:
    def __init__(self, model_name="Qwen/Qwen3-30B-A3B-FP8", llm=None, tokenizer=None):
        """
        챗봇 인스턴스 초기화 (Initialization)
        
        Args:
            model_name (str): 로드할 모델의 경로 또는 Hugging Face ID.
            llm (vllm.LLM, optional): 이미 초기화된 vLLM 엔진 인스턴스. 제공될 경우 새로 로드하지 않음.
            tokenizer (AutoTokenizer, optional): 이미 초기화된 토크나이저 인스턴스.
        """
        print(f"Initializing Chatbot with model: {model_name}...")
        
        # 1. vLLM 엔진 할당 (Dependency Injection)
        # 외부에서 전달된 llm 객체가 있으면 그대로 사용하고, 없으면 새로 생성
        # 이를 통해 반복적인 VRAM 재할당 및 로딩 시간을 방지
        if llm:
            print(">> Existing LLM instance detected. Using the provided engine.")
            self.llm = llm
        else:
            print(">> No LLM instance provided. Loading new engine...")
            self.llm = LLM(
                model=model_name,
                tensor_parallel_size=1,     # 단일 GPU 사용
                gpu_memory_utilization=0.95, # VRAM 점유율 최대화
                dtype="auto" # FP8 모델의 경우 auto 설정 시 config.json에 맞춰 자동 최적화됨
            )

        # 2. 토크나이저 할당
        if tokenizer:
            print(">> Existing Tokenizer instance detected. Using the provided tokenizer.")
            self.tokenizer = tokenizer
        else:
            print(">> Loading new tokenizer...")
            self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

        # 3. 상태 변수 초기화
        self.history = []  # 대화 문맥(Context) 저장소

    def generate_response(self, user_input):
        """
        사용자 입력을 받아 모델의 추론 결과를 반환하고 히스토리를 업데이트합니다.
        """
        # 1. 프롬프트 구성
        # 현재까지의 히스토리에 이번 턴의 사용자 입력을 임시로 추가하여 템플릿을 적용
        current_messages = self.history + [{"role": "user", "content": user_input}]
        
        prompt_str = self.tokenizer.apply_chat_template(
            current_messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # 2. 샘플링 파라미터 설정
        sampling_params = SamplingParams(
            temperature=0.7,
            top_p=0.8,
            max_tokens=4096,
            repetition_penalty=1.05,
        )

        # 3. 추론
        outputs = self.llm.generate([prompt_str], sampling_params)
        raw_output = outputs[0].outputs[0].text.strip()

        # 4. 결과 파싱
        # <think> 태그를 기준으로 내부 사고 과정과 최종 답변을 분리
        thinking_content = ""
        final_response = raw_output

        if "</think>" in raw_output:
            parts = raw_output.split("</think>")
            thinking_content = parts[0].replace("<think>", "").strip()
            if len(parts) > 1:
                final_response = parts[1].strip()
            else:
                final_response = "" # 사고 과정만 출력된 경우 예외 처리

        # 5. 히스토리 업데이트
        # Context Window 관리를 위해 최종 답변만 기록
        self.history.append({"role": "user", "content": user_input})
        self.history.append({"role": "assistant", "content": final_response})

        return {
            "thought": thinking_content,
            "response": final_response,
            "raw": raw_output
        }

    def clear_history(self):
        """대화 내역을 초기화합니다."""
        self.history = []
        print(">> Chat history cleared.")

In [10]:
chatbot = QwenVLLMChatbot(llm=llm, tokenizer=tokenizer)

def print_result(turn, result):
    print(f"\n[Turn {turn} Result]")
    if result["thought"]:
        print(f"[사고]: {result['thought'][:100]}... (생략)")
    print(f"[답변]: {result['response']}")

# Turn 1
q1 = "딸기(Strawberry) 단어에 r이 몇 개 들어있어?"
print(f"\n[입력]: {q1}")
res1 = chatbot.generate_response(q1)
print_result(1, res1)

# Turn 2 (문맥 유지 확인)
q2 = "그럼 'Raspberry'는?"
print(f"\n[입력]: {q2}")
res2 = chatbot.generate_response(q2)
print_result(2, res2)

Initializing Chatbot with model: Qwen/Qwen3-30B-A3B-FP8...
>> Existing LLM instance detected. Using the provided engine.
>> Existing Tokenizer instance detected. Using the provided tokenizer.

[입력]: 딸기(Strawberry) 단어에 r이 몇 개 들어있어?


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


[Turn 1 Result]
[사고]: Okay, let's see. The user is asking how many times the letter 'r' appears in the word "Strawberry." ... (생략)
[답변]: 딸기("Strawberry") 단어에 포함된 **r**의 개수를 확인해보면 다음과 같습니다:

1. **S**  
2. **T**  
3. **R** → 1개  
4. **A**  
5. **W**  
6. **B**  
7. **E**  
8. **R** → 2개  
9. **R** → 3개  
10. **Y**  

따라서 **r은 총 3개** 들어 있습니다.  

**정답:** 3개 ✅

[입력]: 그럼 'Raspberry'는?


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


[Turn 2 Result]
[사고]: Okay, the user asked about the number of 'r's in "Raspberry" after previously asking about "Strawber... (생략)
[답변]: 'Raspberry' 단어에 포함된 **r**의 개수를 확인해보면 다음과 같습니다:

1. **R** → 1개  
2. **A**  
3. **S**  
4. **P**  
5. **B**  
6. **E**  
7. **R** → 2개  
8. **R** → 3개  
9. **Y**  

따라서 **r은 총 3개** 들어 있습니다.  

**정답:** 3개 ✅  

(참고: "Strawberry"와 마찬가지로 "Raspberry"도 3개의 'r'을 가지고 있어요!) 🍓


In [ ]:
chatbot.clear_history()

idx = 1
while True:
    query = input()
    response = chatbot.generate_response(query)
    print_result(idx, response)
    idx += 1